# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
%matplotlib inline
from ipywidgets.embed import embed_minimal_html #To export image to html

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json

import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#pip install gmaps

In [2]:
City_df=pd.read_csv("../WeatherPy/city_data.csv")
City_df.head()

,ID,City Name,Country,Date,MaxTemp,Humidity,lat,lng,Cloudiness,Wind_Speed,Formatted Date
0,5924351,Clyde River,CA,1595731143,39.20,93,70.47,-68.59,75,6.93,2020-07-25
1,1649150,Bengkulu,ID,1595731146,82.74,70,-3.80,102.27,4,7.76,2020-07-25
2,2620871,Hals,DK,1595731146,61.00,94,57.00,10.31,100,3.00,2020-07-25
3,470905,Vuktyl,RU,1595731146,65.10,61,63.86,57.31,57,1.81,2020-07-25
4,3873145,Río Bueno,CL,1595731146,46.40,100,-40.32,-72.97,100,5.82,2020-07-25


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
import gmaps
## Configure gmaps with API key
gmaps.configure(api_key=g_key)



In [4]:
# Store 'Lat' and 'Lng' into  locations
locations=City_df[['lat','lng']]
locations
Humidity_rate=City_df["Humidity"]


# # Create a Humidity Heatmap layer
fig = gmaps.figure(zoom_level=1,center=(10,10))
#fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity_rate, 
                                   dissipating=False, max_intensity=100,
                                   point_radius = 3
                                )
fig.add_layer(heat_layer)
embed_minimal_html('Heatmap.html', views=[fig])

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#new df with all null values dropped
New_City_df=City_df.dropna().reset_index()
New_City_df.head() #reduced to 567 rows

,index,ID,City Name,Country,Date,MaxTemp,Humidity,lat,lng,Cloudiness,Wind_Speed,Formatted Date
0,0,5924351,Clyde River,CA,1595731143,39.20,93,70.47,-68.59,75,6.93,2020-07-25
1,1,1649150,Bengkulu,ID,1595731146,82.74,70,-3.80,102.27,4,7.76,2020-07-25
2,2,2620871,Hals,DK,1595731146,61.00,94,57.00,10.31,100,3.00,2020-07-25
3,3,470905,Vuktyl,RU,1595731146,65.10,61,63.86,57.31,57,1.81,2020-07-25
4,4,3873145,Río Bueno,CL,1595731146,46.40,100,-40.32,-72.97,100,5.82,2020-07-25


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#ideal vacation spots
Hotel_df=New_City_df.loc[((New_City_df["MaxTemp"]>70) & 
                           (New_City_df["MaxTemp"]<75))&
                          (New_City_df["Wind_Speed"]<5)&
                        (New_City_df["Cloudiness"]<10)]
Hotel_df.head()

,index,ID,City Name,Country,Date,MaxTemp,Humidity,lat,lng,Cloudiness,Wind_Speed,Formatted Date
53,53,1516048,Khovd,MN,1595731152,70.12,36,48.01,91.64,3,3.11,2020-07-25
147,148,3407762,Amapá,BR,1595731163,70.99,97,1.00,-52.00,3,1.10,2020-07-25
171,172,3398856,Grajaú,BR,1595731166,73.20,56,-5.82,-46.14,0,2.66,2020-07-25
196,197,6316343,Alta Floresta,BR,1595731169,71.85,55,-9.88,-56.09,0,2.91,2020-07-25
213,214,3388847,São Félix do Xingu,BR,1595731171,71.02,55,-6.64,-51.99,0,1.92,2020-07-25


In [8]:
Hotel_df["Hotel Name"]= ""
Hotel_df.head()

C:\Users\neejo\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,index,ID,City Name,Country,Date,MaxTemp,Humidity,lat,lng,Cloudiness,Wind_Speed,Formatted Date,Hotel Name
53,53,1516048,Khovd,MN,1595731152,70.12,36,48.01,91.64,3,3.11,2020-07-25,
147,148,3407762,Amapá,BR,1595731163,70.99,97,1.00,-52.00,3,1.10,2020-07-25,
171,172,3398856,Grajaú,BR,1595731166,73.20,56,-5.82,-46.14,0,2.66,2020-07-25,
196,197,6316343,Alta Floresta,BR,1595731169,71.85,55,-9.88,-56.09,0,2.91,2020-07-25,
213,214,3388847,São Félix do Xingu,BR,1595731171,71.02,55,-6.64,-51.99,0,1.92,2020-07-25,


In [8]:
base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
lat=Hotel_df["lat"]
lng=Hotel_df["lng"]
params={
    "key":g_key,
    "radius":5000,
    "types":"lodging"
}

for index,row in Hotel_df.iterrows():
    lat=row["lat"]
    lng=row["lng"]
    
    params["location"]=f"{lat},{lng}"
    response=requests.get(base_url,params=params).json()
   # print(json.dumps(response, indent=4, sort_keys=True))
    
    try:
        Hotel_df.loc[index,"Hotel Name"]=response["results"][0]["name"]
    except (KeyError,IndexError):
        print("Missing data.....skipping")
        
Hotel_df.reset_index()        

        
    

C:\Users\neejo\anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\neejo\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing data.....skipping


,level_0,index,ID,City Name,Country,Date,MaxTemp,Humidity,lat,lng,Cloudiness,Wind_Speed,Formatted Date,Hotel Name
0,53,53,1516048,Khovd,MN,1595731152,70.12,36,48.01,91.64,3,3.11,2020-07-25,Hotel Khovd
1,147,148,3407762,Amapá,BR,1595731163,70.99,97,1.00,-52.00,3,1.10,2020-07-25,NaN
2,171,172,3398856,Grajaú,BR,1595731166,73.20,56,-5.82,-46.14,0,2.66,2020-07-25,Hotel Confiança
3,196,197,6316343,Alta Floresta,BR,1595731169,71.85,55,-9.88,-56.09,0,2.91,2020-07-25,Floresta Amazonica Hotel
4,213,214,3388847,São Félix do Xingu,BR,1595731171,71.02,55,-6.64,-51.99,0,1.92,2020-07-25,Apart Hotel Rio Xingu
5,312,313,2509553,Villanueva de la Serena,ES,1595731182,73.00,49,38.98,-5.80,0,1.99,2020-07-25,Hotel Veracruz
6,359,360,2516507,Huéscar,ES,1595731187,73.99,61,37.81,-2.54,0,1.99,2020-07-25,Hotel El Maño
7,467,470,3466980,Caravelas,BR,1595731200,70.72,90,-17.71,-39.25,9,4.41,2020-07-25,Pousada dos Navegantes


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in Hotel_df.iterrows()]
locations = Hotel_df[["lat", "lng"]]


In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig1= gmaps.figure(zoom_level=1,center=(15,15))
fig1.add_layer(marker_layer)
fig1.add_layer(heat_layer)
embed_minimal_html('Hotel_markers.html', views=[fig1])

# Display figure
fig1


Figure(layout=FigureLayout(height='420px'))